In [ ]:
%tensorflow_version 1.x
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

TensorFlow 1.x selected.
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.go

In [ ]:
%tensorflow_version 1.x
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/bert-paper/")

TensorFlow 1.x selected.
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.85.134.26:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6424815034630768082),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13657108064386232939),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14215918783206324855),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1234333259306256863),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 15743690873476514698),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 9686652356961795513),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 7097014890035849659),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 9288874256622579385),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 4371729254516

In [ ]:
FEATURES = ["title", "keywords", "abstract"]
OUTDIR = "gs://paper/scibert-cs-{}/".format("+".join(FEATURES)) # Google cloud datastorage
PRETRAINED_MODEL_DIR = "gs://bert-eng/scibert_scivocab_uncased" # google cloud model
DATA_DIR = "../data/" # Local data directory

if "abstract" in FEATURES:
    max_seq_length = 512
elif len(FEATURES) == 2:
    max_seq_length = 256
else:
    max_seq_length = 128
FEATURES = ",".join(FEATURES)

In [ ]:
import os
if "keywords" in FEATURES:
    checkpoint = os.path.join(OUTDIR, "model.ckpt-686")
else:
    checkpoint = os.path.join(OUTDIR, "model.ckpt-730")
checkpoint

In [ ]:
!python3 run_cs.py \
    --data_dir=$DATA_DIR \
    --features=$FEATURES \
    --bert_config_file=$PRETRAINED_MODEL_DIR/bert_config.json \
    --vocab_file=$PRETRAINED_MODEL_DIR/vocab.txt \
    --output_dir=$OUTDIR \
    --do_lower_case=True \
    --max_seq_length=$max_seq_length \
    --do_predict=True \
    --predict_checkpoint_path=$checkpoint \
    --predict_batch_size=128 \
    --use_tpu=True \
    --tpu_name=$TPU_ADDRESS

In [ ]:
import os
import numpy as np
lines = []
for line in tf.gfile.Open(os.path.join(OUTDIR, "test_results.tsv")):
    lines.append(line.strip())    
probabilities = np.empty((len(lines), len(lines[0].split())))

for ii, line in enumerate(lines):
    probabilities[ii,:] = list(map(float, line.split()))
# np.save("cs-scibert-{}-valid.npy".format(FEATURES.replace(",","+")), probabilities)

from run_cs import PaperProcessor
prosessor = PaperProcessor(FEATURES.split(","))
journal_2_index = {journal: i for i, journal in enumerate(prosessor.get_labels())}

import json
y_valid = []
for line in prosessor._read_jsonl(os.path.join(DATA_DIR, "cs-valid.jsonl")):
    y_valid.append(journal_2_index[line['journal'].lower()])
y_valid = np.asarray(y_valid)   

print(np.mean(np.repeat(y_valid, 1).reshape(-1,1) == np.argsort(probabilities, axis=-1)[:,:-2:-1])*1)
print(np.mean(np.repeat(y_valid, 3).reshape(-1,3) == np.argsort(probabilities, axis=-1)[:,:-4:-1])*3)
print(np.mean(np.repeat(y_valid, 5).reshape(-1,5) == np.argsort(probabilities, axis=-1)[:,:-6:-1])*5)
print(np.mean(np.repeat(y_valid, 10).reshape(-1,10) == np.argsort(probabilities, axis=-1)[:,:-11:-1])*10)


0.47888531618435154
0.7652733118971062
0.8647374062165059
0.9459807073954984
